In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.__version__

'2.0.0'

In [3]:
tf.test.is_gpu_available()

True

In [4]:
image_filenames = glob.glob('./dc/train/*.jpg')
image_testnames = glob.glob('./dc/test/*.jpg')

In [5]:
len(image_testnames)

12500

In [6]:
image_filenames = np.random.permutation(image_filenames)
image_testnames = np.random.permutation(image_testnames)

In [7]:
train_labels = list(map(lambda x: float(x.split('\\')[1].split('.')[0] == 'cat'),image_filenames ))   #lambda表达式和map全局函数
test_labels = list(map(lambda x: float(x.split('\\')[1].split('.')[0] == 'cat'),image_testnames ))   #lambda表达式和map全局函数

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((image_filenames, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((image_testnames, test_labels))

In [9]:
dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.float32)>

In [10]:
def _pre_read(image_filenames, label):
    image = tf.io.read_file(image_filenames)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,(200,200))
    image = tf.reshape(image, [200,200,1])
    image = tf.image.per_image_standardization(image)
    label = tf.reshape(label, [1])
    return image, label

In [11]:
dataset = dataset.map(_pre_read)
test_dataset = test_dataset.map(_pre_read)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [12]:
dataset

<MapDataset shapes: ((200, 200, 1), (1,)), types: (tf.float32, tf.float32)>

In [13]:
dataset = dataset.shuffle(200)
test_dataset = test_dataset.shuffle(200)

In [14]:
dataset = dataset.repeat()
dataset = dataset.batch(32)

In [15]:
dataset

<BatchDataset shapes: ((None, 200, 200, 1), (None, 1)), types: (tf.float32, tf.float32)>

In [16]:
model = keras.Sequential()

In [17]:
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape = (200, 200, 1)))

In [18]:
model.add(layers.MaxPooling2D((2,2)))

In [19]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [20]:
model.add(layers.MaxPooling2D((2,2)))

In [21]:
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [22]:
model.add(layers.MaxPooling2D((2,2)))

In [23]:
model.add(layers.Flatten())

In [24]:
model.add(layers.Dense(64, activation = 'relu'))

In [25]:
model.add(layers.Dense(1, activation = 'sigmoid'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0

In [27]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['acc']   #其他loss函数'mse', 'mae'
)

In [29]:
history = model.fit(dataset, epochs=10, steps_per_epoch=781, validation_data=test_dataset)

Train for 781 steps, validate for 12500 steps
Epoch 1/10
781/781 [==============================] - 317s 406ms/step - loss: 0.4073 - acc: 0.8138


ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (200, 200, 1)

In [ ]:
plt.plot(history.epoch, history.history['loss'], 'r')
plt.plot(history.epoch, history.history['val_loss'], 'b')

In [ ]:
plt.plot(history.epoch, history.history['acc'], 'r')
plt.plot(history.epoch, history.history['val_acc'], 'b')